In [5]:
import pandas
from simpletransformers.seq2seq import Seq2SeqModel,Seq2SeqArgs

In [17]:
#set the required parameters for the model to be trained on
m_args = Seq2SeqArgs()
m_args.num_train_epochs = 20
m_args.no_save = False
m_args.evaluate_generated_text = True
m_args.evaluate_during_training = True
m_args.evaluate_during_training_verbose = True
m_args.overwrite_output_dir = True
m_args.use_multiprocessed_decoding = True
m_args.use_multiprocessing_for_evaluation = True
m_args.save_model_every_epoch = False
m_args.max_length = 200
m_args.save_best_model = True
m_args.save_steps = 5080

#Instantiate and Initialize a seq2seq model with these args
model_obj = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-large",
    args=m_args,
    use_cuda=True
)

In [8]:
import re
from data_manipulation import data_formatting

In [9]:
raw_base_data_set, raw_summary_data_set = data_formatting.load_dataset(file_path="dataset/batch1.csv", encoding_standard="iso-8859-1")
cleaned_base_set = data_formatting.clean_dataset(raw_base_data_set)
cleaned_summary_data_set = data_formatting.clean_dataset(raw_summary_data_set)


d:\GitHub\News-Article-Text-Summarizer\data_manipulation\data_formatting.py:5: DtypeWarning: Columns (2,3,4,5,6,7,8,9,11,12,13,14,15,16,17,18,19,21,22,24,25,26,28,29,30,31,32,33,34,35,36,37,38,39,40,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259

In [10]:
#setting up the data as required by the model
num_data_pairs = len(cleaned_base_set)
num_data_pairs

4516

In [11]:
import math
training_data_set_size = math.floor(num_data_pairs * 0.9)
training_data_set_size

eval_data_set_size = len(cleaned_base_set) - training_data_set_size
#eval_data_set_size = 10 - training_data_set_size
eval_data_set_size

452

In [12]:
#create training data set
training_data_set = []
for i in range(training_data_set_size):
    new_list = [cleaned_base_set[i], cleaned_summary_data_set[i]]
    training_data_set.append(new_list)

In [13]:
#create evaluation data set
evaluation_data_set = []
for i in range(eval_data_set_size):
    new_list = [cleaned_base_set[training_data_set_size + i], cleaned_summary_data_set[training_data_set_size + i]]
    evaluation_data_set.append(new_list)

In [14]:
#create a pandas training data frame
training_data_frame = pandas.DataFrame(
    training_data_set, columns=["input_text","target_text"]
)

#create a pandas evaluation data frame
evaluation_data_frame = pandas.DataFrame(
    evaluation_data_set, columns=["input_text","target_text"]
)

In [ ]:
import gc
import torch
gc.collect
torch.cuda.empty_cache()

In [18]:
#training the model
model_obj.train_model(
    training_data_frame, 
    eval_data = evaluation_data_frame, 
    output_dir="outputs/"
)

  0%|          | 0/4064 [00:00<?, ?it/s]

c:\Users\neeli\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Running Epoch 0 of 20:   0%|          | 0/508 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 4.00 GiB total capacity; 2.63 GiB already allocated; 8.20 MiB free; 2.65 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF